# Ingest GitHub Issues into Qdrant

This sample shows how to get started loading and querying dotnet/runtime repo GitHub issue embeddings using Azure OpenAI and the Qdrant SDK

## Install packages

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.9"
#r "nuget: Qdrant.Client, 1.6.0-alpha.1"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.9 Qdrant.Client, 1.6.0-alpha.1

In [2]:
#r "nuget: Octokit, 9.0.0"
#r "nuget: Octokit.Reactive, 9.0.0"

Installed Packages Octokit, 9.0.0 Octokit.Reactive, 9.0.0

In [3]:
#r "nuget:Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23557.4"

Installed Packages Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23557.4

Loading extension script from `/home/codespace/.nuget/packages/microsoft.dotnet.interactive.aiutilities/1.0.0-beta.23557.4/interactive-extensions/dotnet/extension.dib`

## Add using statements

In [4]:
using Azure;
using Azure.AI.OpenAI;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.AIUtilities;
using Octokit;

## Configure Azure OpenAI credentials

In [5]:
var azureOpenAIKey = await Kernel.GetPasswordAsync("Provide your OPEN_AI_KEY");
var azureOpenAIEndpoint = await Kernel.GetInputAsync("Provide the OPEN_AI_ENDPOINT");
var embeddingDeployment = await Kernel.GetInputAsync("Provide embedding name");

## Configure GitHub credentials 

You will need access token with rights to query and update issues.

In [7]:
var githubKey = await Kernel.GetPasswordAsync("Provide your Github api key");
var repoName = await Kernel.GetInputAsync("Provide repo");
var org = await Kernel.GetInputAsync("Provide org");

## Configure OpenAI client

In [8]:
OpenAIClient openAIClient = new (new System.Uri(azureOpenAIEndpoint), new AzureKeyCredential(azureOpenAIKey.GetClearTextPassword()));

## Configure GitHub client

In [9]:
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));

if (!string.IsNullOrEmpty(githubKey.GetClearTextPassword())) {
    Console.WriteLine("Using GitHub API token");
    var tokenAuth = new Credentials(githubKey.GetClearTextPassword());
    gitHubClient.Credentials = tokenAuth;
} else {
    Console.WriteLine("Using anonymous GitHub API");
}

Using GitHub API token


## Download data from GitHub

### Get labels from the repository

In [10]:
var allLabels = await gitHubClient.Issue.Labels.GetAllForRepository(org, repoName);

In [11]:
var areaLabels = allLabels.Where(label => label.Name.StartsWith("area-", StringComparison.OrdinalIgnoreCase)).ToList();

### Get all issues from the repository

In [12]:
var allIssues = new List<Issue>();

In [13]:
foreach(var label in areaLabels)
{
    var request = new RepositoryIssueRequest
    {
        Filter = IssueFilter.All
    };
    
    request.Labels.Add(label.Name);

    var apiOptions = new ApiOptions
    {
        PageSize = 50,
        PageCount = 1
    };

    var issues = await gitHubClient.Issue.GetAllForRepository(org, repoName, request, apiOptions);
    
    allIssues.AddRange(issues);
}

In [14]:
allIssues.Count()

4137

In [15]:
public record GitHubIssue(string Title, string Text, string Area, string Url);

In [16]:
var dataCollection = 
    allIssues
        .DistinctBy(issue => issue.Id)
        .Select(issue => 
            new GitHubIssue(
                issue.Title,
                issue.Body,
                issue.Labels?.Where(l => 
                    l.Name.StartsWith("area-",StringComparison.OrdinalIgnoreCase))
                        .FirstOrDefault()?
                        .Name?
                        .Replace("area-",string.Empty)?
                        .Replace("-"," "),
                issue.HtmlUrl));

## Helper functions to save and load to disk

In [17]:
using System.IO;
using System.Text.Json;
using System.Text.Json.Serialization;

public async Task SaveIssuesToFileAsync(IEnumerable<GitHubIssue> data, string fileName)
{ 
    var filePath = Path.Combine("..","..","..","Data",fileName);
    var issuesJson = JsonSerializer.Serialize(data,new JsonSerializerOptions(JsonSerializerOptions.Default){WriteIndented=true});
    await File.WriteAllTextAsync(filePath, issuesJson);
}

public async Task<GitHubIssue[]> LoadIssuesFromFileAsync(string fileName)
{
    var filePath = Path.Combine("..","..","..","Data",fileName);
    var text = await File.ReadAllTextAsync(filePath);
    return JsonSerializer.Deserialize<GitHubIssue[]>(text);
}

In [18]:
await SaveIssuesToFileAsync(dataCollection, "issues.json");

## Naive Search

In [19]:
var dataCollection = await LoadIssuesFromFileAsync("issueWithEmbeddings.json");

In [20]:
public string[] NaiveSearch(string query, IEnumerable<GitHubIssue> data,int resultLimit = 1)
{
    return data
            .Where(d => d.Text?.Contains(query)==true)
            .Select(d => d.Text)
            .Take(resultLimit)
            .ToArray();
}

In [21]:
NaiveSearch("What are the latest issues for AOT",dataCollection).Display();

[ ]

## Chunk issues

In [22]:
var dataCollection = await LoadIssuesFromFileAsync("issues.json");

In [23]:
dataCollection.Count()

4128

### Initialize collection of issues with chunks

In [24]:
public record TextWithEmbedding(string Text, float[] Embedding);
public record IssueWithChunks(GitHubIssue Issue,List<TextWithEmbedding> Chunks);

In [25]:
var issuesWithChunksCollection = 
    dataCollection
        .Select(issue => new IssueWithChunks(issue, new ()))
        .ToArray();

### Helper functions to save and load chunks to disk

In [26]:
using System.IO;
using System.Text.Json;
using System.Text.Json.Serialization;

public async Task SaveIssuesWithChunksToFileAsync(IEnumerable<IssueWithChunks> data, string fileName)
{ 
    var filePath = Path.Combine("..","..","..","Data",fileName);
    var issuesJson = JsonSerializer.Serialize(data,new JsonSerializerOptions(JsonSerializerOptions.Default){WriteIndented=true});
    await File.WriteAllTextAsync(filePath, issuesJson);
}

public async Task<IssueWithChunks[]> LoadIssuesWithChunksFromFileAsync(string fileName)
{
    var filePath = Path.Combine("..","..","..","Data",fileName);
    var text = await File.ReadAllTextAsync(filePath);
    return JsonSerializer.Deserialize<IssueWithChunks[]>(text);
}

### Chunk data and generate embeddings

In [27]:
var tokenizer = await Tokenizer.CreateAsync(TokenizerModel.ada2);

var counter = 0;

foreach(var item in issuesWithChunksCollection.Take(100))
{
    var fullText = item.Issue.Text;
    if(string.IsNullOrWhiteSpace(fullText))
        continue;

    var chunks = 
        tokenizer
            .ChunkByTokenCountWithOverlap(fullText, 3000, 50)
            .Select(t => 
$"""
Title: {item.Issue.Title}
Area: {item.Issue.Area}

{t}
""")
            .Chunk(16)
            .ToArray();

    foreach(var chunk in chunks)
    {
        
        var embeddingResponse = await openAIClient.GetEmbeddingsAsync(new EmbeddingsOptions(embeddingDeployment,chunk));
        item.Chunks.AddRange(
            embeddingResponse.Value.Data.Select(d => 
                new TextWithEmbedding(chunk[d.Index],d.Embedding.ToArray())));
    }

    if(counter % 50 == 0)
        await SaveIssuesWithChunksToFileAsync(issuesWithChunksCollection, "areaIssuesWithEmbeddingsSubset.json");
    counter++;
}

await SaveIssuesWithChunksToFileAsync(issuesWithChunksCollection, "areaIssuesWithEmbeddingsSubset.json");

In [29]:
issuesWithChunksCollection.Take(5).DisplayTable();

Issue Chunks GitHubIssue { Title = Make it easy to determine entry assembly path when debugging CoreCLR, Text = When debugging for example crash dumps of .NET applications, it is often helpful to be able to quickly identify the entry assembly for the process/dump. Currently this information is easily accessible ... Title Make it easy to determine entry assembly path when debugging CoreCLR Text When debugging for example crash dumps of .NET applications, it is often helpful to be able to quickly identify the entry assembly for the process/dump. Currently this information is easily accessible from managed code via `Assembly.GetEntryAssembly().Location` but it's very difficult to get to when in the debugger without ability to invoke methods or run managed code in general. For example, if the process was started via `dotnet app.dll`, it's difficult to determine the `app.dll` from a dump.

The proposal is to add a new static global variable into coreclr module, which will contain the path of the entry assembly once that information is known.

In detail:
* Add static global variable with the full path to the entry assembly
* The variable would be a `WCHAR *` type - UTF16 encoded string stored on the heap
* Modify `EnumMemRegions` such that string on the heap is stored in a dump even if the dump will not contain the full heap
* The variable is only initialized if the runtime "executes an assembly", so not if the runtime was started via native hosting APIs and is invoked using `get_delegate` or similar mechanisms.
* The variable is left `NULL` until it can be initialized, or for the lifetime of the runtime if there's no entry assembly defined (native hosting scenarios).
* The variable doesn't change its value once it's set to some path

To be figured out:
* The name of the variable
* Interaction with startup hooks - when exactly we would set the value (before or after startup hooks) Area AssemblyLoader coreclr Url https://github.com/dotnet/runtime/issues/94474 index value 0 TextWithEmbedding { Text = Title: Make it easy to determine entry assembly path when debugging CoreCLR\nArea: AssemblyLoader coreclr\n\nWhen debugging for example crash dumps of .NET applications, it is often helpful to be able to quickly identify the entry assembly for the process/dump. Currently t... Text Title: Make it easy to determine entry assembly path when debugging CoreCLR
Area: AssemblyLoader coreclr

When debugging for example crash dumps of .NET applications, it is often helpful to be able to quickly identify the entry assembly for the process/dump. Currently this information is easily accessible from managed code via `Assembly.GetEntryAssembly().Location` but it's very difficult to get to when in the debugger without ability to invoke methods or run managed code in general. For example, if the process was started via `dotnet app.dll`, it's difficult to determine the `app.dll` from a dump.

The proposal is to add a new static global variable into coreclr module, which will contain the path of the entry assembly once that information is known.

In detail:
* Add static global variable with the full path to the entry assembly
* The variable would be a `WCHAR *` type - UTF16 encoded string stored on the heap
* Modify `EnumMemRegions` such that string on the heap is stored in a dump even if the dump will not contain the full heap
* The variable is only initialized if the runtime "executes an assembly", so not if the runtime was started via native hosting APIs and is invoked using `get_delegate` or similar mechanisms.
* The variable is left `NULL` until it can be initialized, or for the lifetime of the runtime if there's no entry assembly defined (native hosting scenarios).
* The variable doesn't change its value once it's set to some path

To be figured out:
* The name of the variable
* Interaction with startup hooks - when exactly we would set the value (before or after startup hooks) Embedding [ -0.006981608, 0.022468084, 0.0102

In [32]:
await SaveIssuesWithChunksToFileAsync(issuesWithChunksCollection, "areaIssuesWithEmbeddingsSubset.json");

## Embedding Search

In [33]:
#r "nuget: System.Numerics.Tensors, 8.0.0-rc.2.23479.6"

Installed Packages System.Numerics.Tensors, 8.0.0-rc.2.23479.6

In [34]:
using System.Numerics.Tensors;

In [35]:
public class SimilarityComparer : ISimilarityComparer<float[]>
{
    public float Score(float[] a, float[] b)
    {
        return TensorPrimitives.CosineSimilarity(a,b);
    }
}

In [36]:
public async Task<string[]> EmbeddingSearchAsync(string query, IEnumerable<IssueWithChunks> data,int resultLimit = 1)
{
    var embeddingResponse = await openAIClient.GetEmbeddingsAsync(new EmbeddingsOptions(embeddingDeployment,new [] {query}));
    var embeddingVector = embeddingResponse.Value.Data[0].Embedding.ToArray();

    var searchResults = 
        data
        .SelectMany(d => d.Chunks)
        .ScoreBySimilarityTo(embeddingVector,new SimilarityComparer(),c => c.Embedding)
        .OrderByDescending(e => e.Value)
        .Where(e => e.Value > 0.5)
        .Take(resultLimit)
        .Select(e => e.Key.Text)
        .ToArray();
    
    return searchResults;
}

In [38]:
(await EmbeddingSearchAsync("What are the latest issues for AOT", issuesWithChunksCollection, 3)).Display();

[ Title: [docs] Add Mono Library Mode and update Mono Profiled AOT
Area: Build mono

This PR looks to add/update documentation on [Mono's Library Mode](https://github.com/dotnet/runtime/issues/79377) and [Mono's Profiled AOT](https://github.com/dotnet/runtime/issues/69512) workflows., Title: [browser][AOT] Space in path to published project on unix
Area: Build mono

Theoretically, enclosing included directories in quotations should be enough to escape spaces in Unix.

Reason: wbt use `artifacts/bin/dotnet-latest/packs/Microsoft.NET.Runtime.Emscripten.3.1.34.Sdk.linux-x64/9.0.0-alpha.1.23528.4/tools/bin/clang` that is `16.0.5`. Version `17.0.0` is already working fine on Unix, should not need any code edition, only unification of `wbt` -> `wbt artifacts` for all the platforms.
The issue reported clang `15.0.0` https://github.com/dotnet/runtime/issues/92335, Title: [wasm] WBT `SatelliteAssembliesTests.CheckThatSatelliteAssembliesAreNotAOTed` failing
Area: Build mono

Test failing with [rolling build](https://dev.azure.com/dnceng-public/public/_build/results?buildId=371479&view=results):

```
 [4/5] System.Private.CoreLib.dll.bc -> System.Private.CoreLib.dll.o 

WasmApp.Native.targets(379,5): error : Failed to compile /root/helix/work/workitem/e/wbt/ddpnbu30_we1/obj/Release/net8.0/browser-wasm/wasm/for-publish/aot-instances.dll.bc -> /root/helix/work/workitem/e/wbt/ddpnbu30_we1/obj/Release/net8.0/browser-wasm/wasm/for-publish/aot-instances.dll.o 
WasmApp.Native.targets(379,5): error : emcc: warning: linker setting ignored during compilation: 'EXPORT_ES6' [-Wunused-command-line-argument] 
WasmApp.Native.targets(379,5): error : Killed
```

`WasmApp.Native.targets(379,5): error : emcc: error: '/root/helix/work/workitem/e/dotnet-latest/packs/Microsoft.NET.Runtime.Emscripten.3.1.34.Sdk.linux-x64/8.0.0-rc.1.23411.1/tools/bin/clang++ -target wasm32-unknown-emscripten -fvisibility=default -mllvm -combiner-global-alias-analysis=false -mllvm -wasm-enable-sjlj -mllvm -disable-lsr -DEMSCRIPTEN --sysroot=/root/helix/work/workitem/e/dotnet-latest/packs/Microsoft.NET.Runtime.Emscripten.3.1.34.Cache.linux-x64/8.0.0-rc.1.23411.1/tools/emscripten/cache/sysroot -Xclang -iwithsysroot/include/fakesdl -Xclang -iwithsysroot/include/compat -msimd128 -O0 -g3 -fwasm-exceptions -c /root/helix/work/workitem/e/wbt/ddpnbu30_we1/obj/Release/net8.0/browser-wasm/wasm/for-publish/aot-instances.dll.bc -o /tmp/tmpeo8pzM.tmp' failed (returned 137) [took 128.818s] `

[Changes since last passing rolling build](https://github.com/dotnet/runtime/compare/64a67710163...55828b9aa91).

This includes commits from @pavelsavara @ivanpovazan @vargaz @radekdoulik 
<!-- Error message template -->
### Known Issue Error Message

Fill the error message using [step by step known issues guidance](https://github.com/dotnet/arcade/blob/main/Documentation/Projects/Build%20Analysis/KnownIssues.md#how-to-fill-out-a-known-issue-error-section).

<!-- Use ErrorMessage for String.Contains matches. Use ErrorPattern for regex matches (single line/no backtracking). Set BuildRetry to `true` to retry builds with this error. Set ExcludeConsoleLog to `true` to skip helix logs analysis. -->

```json
{
 "ErrorMessage": "error : Failed to compile ",
 "ErrorPattern": "",
 "BuildRetry": false,
 "ExcludeConsoleLog": false
}
```


<!--Known issue error report start -->
### Report

|Build|Definition|Test|Pull Request|
|---|---|---|---|
|[2242043](https://dev.azure.com/dnceng/internal/_build/results?buildId=2242043)|dotnet-runtime|[wasmaot.x64.micro.net8.0.Partition27.WorkItemExecution](https://dev.azure.com/dnceng//internal/_build/results?buildId=2242043&view=ms.vss-test-web.build-test-results-tab&runId=51894143&resultId=100058)||
|[371479](https://dev.azure.com/dnceng-public/public/_build/results?buildId=371479)|dotnet/runtime|[Workloads-Wasm.Build.Tests.SatelliteAssembliesTests.WorkItemExecution](https://dev.azure.com/dnceng-public/public/_build/results?buildId=3714

## Store in DB

### Start DB locally

In [39]:
docker run -d -p 6333:6333 -p 6334:6334 -v "$pwd/qdrant_storage:/qdrant/storage:z" qdrant/qdrant

1de9918c3fb2ef284da97d2da45f44d24006b3dd9ab956e7bec6883d9d245d8a


In [40]:
using Qdrant.Client;
using Qdrant.Client.Grpc;

### Initialize Qdrant client

In [41]:
var qdrantClient = new QdrantClient(host: "localhost",port: 6334,https:false);

### Create collection

In [42]:
var collectionName = "gh_issues";

In [43]:
var collections = await qdrantClient.ListCollectionsAsync();
if(collections.Where(x => x.Contains(collectionName)).Count() > 0) 
    await qdrantClient.DeleteCollectionAsync(collectionName);

In [44]:
await qdrantClient.CreateCollectionAsync(collectionName, new VectorParams { Size=1536, Distance=Distance.Cosine})

### Map issue embeddings to points

In [45]:
var vectors = 
    issuesWithChunksCollection
        .Where(d => d.Chunks.Count > 0)
        .SelectMany(d => 
            d.Chunks.Select(c => new {
                Embedding=c.Embedding,
                Text=$"<issueTitle>{d.Issue.Title}</issueTitle>\n<issueUrl>{d.Issue.Url}</issueUrl>\n<issueArea>{d.Issue.Area}</issueArea>\n<issueSnippet>{c.Text}</issueSnippet>"
                }))
        .ToList();
  

In [46]:
var points = vectors.Select(vector => 
{
    var point = new PointStruct
    {
        Id = new PointId { Uuid = Guid.NewGuid().ToString() },
        Vectors = vector.Embedding,
        Payload = 
            {
                ["text"] = vector.Text
            }
    };
    return point;
}).ToList();


### Insert data into Qdrant collection

In [47]:
await qdrantClient.UpsertAsync(collectionName,points);

## Get Count

In [48]:
await qdrantClient.CountAsync(collectionName)

95

## Search with Qdrant

In [49]:
public async Task<string[]> SearchWithQdrantAsync(string query, string collectionName, int resultLimit = 1)
{
    var embeddingResponse = await openAIClient.GetEmbeddingsAsync(new EmbeddingsOptions(embeddingDeployment,new [] {query}));
    var embeddingVector = embeddingResponse.Value.Data[0].Embedding.ToArray();

    var results = await qdrantClient.SearchAsync(collectionName,embeddingVector, limit:(ulong)resultLimit);
    return results.Select(r => r.Payload["text"].StringValue).ToArray();
}

In [50]:
(await SearchWithQdrantAsync("What are the latest issues for AOT", collectionName, 3)).Display();

[ <issueTitle>[docs] Add Mono Library Mode and update Mono Profiled AOT</issueTitle>
<issueUrl>https://github.com/dotnet/runtime/pull/94578</issueUrl>
<issueArea>Build mono</issueArea>
<issueSnippet>Title: [docs] Add Mono Library Mode and update Mono Profiled AOT
Area: Build mono

This PR looks to add/update documentation on [Mono's Library Mode](https://github.com/dotnet/runtime/issues/79377) and [Mono's Profiled AOT](https://github.com/dotnet/runtime/issues/69512) workflows.</issueSnippet>, <issueTitle>[browser][AOT] Space in path to published project on unix</issueTitle>
<issueUrl>https://github.com/dotnet/runtime/pull/94306</issueUrl>
<issueArea>Build mono</issueArea>
<issueSnippet>Title: [browser][AOT] Space in path to published project on unix
Area: Build mono

Theoretically, enclosing included directories in quotations should be enough to escape spaces in Unix.

Reason: wbt use `artifacts/bin/dotnet-latest/packs/Microsoft.NET.Runtime.Emscripten.3.1.34.Sdk.linux-x64/9.0.0-alpha.1.23528.4/tools/bin/clang` that is `16.0.5`. Version `17.0.0` is already working fine on Unix, should not need any code edition, only unification of `wbt` -> `wbt artifacts` for all the platforms.
The issue reported clang `15.0.0` https://github.com/dotnet/runtime/issues/92335</issueSnippet>, <issueTitle>[wasm] WBT `SatelliteAssembliesTests.CheckThatSatelliteAssembliesAreNotAOTed` failing</issueTitle>
<issueUrl>https://github.com/dotnet/runtime/issues/90458</issueUrl>
<issueArea>Build mono</issueArea>
<issueSnippet>Title: [wasm] WBT `SatelliteAssembliesTests.CheckThatSatelliteAssembliesAreNotAOTed` failing
Area: Build mono

Test failing with [rolling build](https://dev.azure.com/dnceng-public/public/_build/results?buildId=371479&view=results):

```
 [4/5] System.Private.CoreLib.dll.bc -> System.Private.CoreLib.dll.o 

WasmApp.Native.targets(379,5): error : Failed to compile /root/helix/work/workitem/e/wbt/ddpnbu30_we1/obj/Release/net8.0/browser-wasm/wasm/for-publish/aot-instances.dll.bc -> /root/helix/work/workitem/e/wbt/ddpnbu30_we1/obj/Release/net8.0/browser-wasm/wasm/for-publish/aot-instances.dll.o 
WasmApp.Native.targets(379,5): error : emcc: warning: linker setting ignored during compilation: 'EXPORT_ES6' [-Wunused-command-line-argument] 
WasmApp.Native.targets(379,5): error : Killed
```

`WasmApp.Native.targets(379,5): error : emcc: error: '/root/helix/work/workitem/e/dotnet-latest/packs/Microsoft.NET.Runtime.Emscripten.3.1.34.Sdk.linux-x64/8.0.0-rc.1.23411.1/tools/bin/clang++ -target wasm32-unknown-emscripten -fvisibility=default -mllvm -combiner-global-alias-analysis=false -mllvm -wasm-enable-sjlj -mllvm -disable-lsr -DEMSCRIPTEN --sysroot=/root/helix/work/workitem/e/dotnet-latest/packs/Microsoft.NET.Runtime.Emscripten.3.1.34.Cache.linux-x64/8.0.0-rc.1.23411.1/tools/emscripten/cache/sysroot -Xclang -iwithsysroot/include/fakesdl -Xclang -iwithsysroot/include/compat -msimd128 -O0 -g3 -fwasm-exceptions -c /root/helix/work/workitem/e/wbt/ddpnbu30_we1/obj/Release/net8.0/browser-wasm/wasm/for-publish/aot-instances.dll.bc -o /tmp/tmpeo8pzM.tmp' failed (returned 137) [took 128.818s] `

[Changes since last passing rolling build](https://github.com/dotnet/runtime/compare/64a67710163...55828b9aa91).

This includes commits from @pavelsavara @ivanpovazan @vargaz @radekdoulik 
<!-- Error message template -->
### Known Issue Error Message

Fill the error message using [step by step known issues guidance](https://github.com/dotnet/arcade/blob/main/Documentation/Projects/Build%20Analysis/KnownIssues.md#how-to-fill-out-a-known-issue-error-section).

<!-- Use ErrorMessage for String.Contains matches. Use ErrorPattern for regex matches (single line/no backtracking). Set BuildRetry to `true` to retry builds with this error. Set ExcludeConsoleLog to `true` to skip helix logs analysis. -->

```json
{
 "ErrorMessage": "error : Failed to compile ",
 "ErrorPattern": "",
 "BuildRetry": false,
 "ExcludeConsoleLog": false
}
```


<!-